https://gist.github.com/IT102Gists/aa31be3ac569878cc7df3a8b37ae25a4

In [1]:
# import libraries
import json
from urllib.request import urlopen
import numpy as np
import pandas as pd

In [2]:
# set api url
api = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

In [3]:
# define funtion, if clauses added to bypass missing values
def locate_data (books):
    data = []  
    for isbn in books: 
        resp = urlopen(api + isbn)
        book_data = json.load(resp)
        volume_info = book_data["items"][0]["volumeInfo"]
        author = volume_info["authors"]
        prettify_author = author if len(author) > 1 else author[0]
        if "averageRating" in volume_info:
            volume_info['averageRating']
        else: 
            volume_info['averageRating'] =np.nan

        if "ratingsCount" in volume_info:
            volume_info['ratingsCount']
        else: 
            volume_info['ratingsCount'] =np.nan


        data.append({"ISBN": isbn, 
                "Title" :volume_info['title'], 
                "Author": prettify_author,
                "Page Count": volume_info['pageCount'],
                "Publication Date" : volume_info['publishedDate'],
                "Language" : volume_info["language"], 
                 "AverageRating": volume_info['averageRating'], 
                     "RatingsCount":volume_info['ratingsCount'],
                    })
    return data

In [4]:
# import csv of isbn
raw_data = pd.read_csv("ISBN Data first 100 - Sheet1.csv")
raw_data["ISBN"]= raw_data["ISBN"].astype(str)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ISBN     104 non-null    object
 1   Status   104 non-null    object
 2   Date     48 non-null     object
dtypes: object(3)
memory usage: 2.6+ KB


In [6]:
# due to internet limitations, gather data in sections and store in dataframe
df1_25 = pd.DataFrame(locate_data(raw_data["ISBN"][:25]))

In [7]:
df25_50 = pd.DataFrame(locate_data(raw_data["ISBN"][25:50]))

In [8]:
df50_75 = pd.DataFrame(locate_data(raw_data["ISBN"][50:75]))

In [9]:
df75_100 = pd.DataFrame(locate_data(raw_data["ISBN"][75:100]))

In [10]:
df100_104 = pd.DataFrame(locate_data(raw_data["ISBN"][100:]))

In [11]:
# set isbn to index
df1_25.set_index("ISBN", inplace = True)
df25_50.set_index("ISBN", inplace = True)
df50_75.set_index("ISBN", inplace = True)
df75_100.set_index("ISBN", inplace = True)
df100_104.set_index("ISBN", inplace = True)

In [12]:
# join the list together
API_Data1 = [df1_25, df25_50,df50_75,df75_100, df100_104]
Books1_104 = pd.concat(API_Data1)

In [13]:
#view dataframe
Books1_104.head()

,Title,Author,Page Count,Publication Date,Language,AverageRating,RatingsCount
ISBN,,,,,,,
4048692712,ちょっと今から仕事やめてくる,北川恵海,235,2015-02-25,ja,3.0,9.0
4048659502,ヒーローズ(株)!!!,北川恵海,329,2016-04,ja,5.0,1.0
404870138X,0能者ミナト,葉山透,341,2011-02,ja,NaN,NaN
4102134034,シャーロック・ホームズの思い出,"ドイル,A.(アーサー)C.",350,1989,ja,NaN,NaN
4102134069,四つの署名,"ドイル,A.(アーサー)C.",196,1993,ja,3.0,1.0


In [14]:
#save as csv
Books1_104.to_csv("Books1_104.csv")